In [6]:
import pandas as pd
import random
import faker

# Initialize Faker for generating random data
fake = faker.Faker()

# Generate random data with some invalid entries
data = [
    [fake.email(), fake.name(), fake.phone_number()],
    [fake.email(), fake.name(), fake.phone_number()],
    [fake.email(), fake.name(), fake.phone_number()],
    [fake.email(), fake.name(), fake.phone_number()],
    ['invalid_email', 'John Doe', '+1234567890'],  # Invalid email
    ['jane.smith@example.com', 'Jane Smith', 'not_a_phone_number'],  # Invalid phone
    ['alice.johnson@example.com', 'Alice Johnson', '+1123456789'],
    ['bob.brown@example.com', 'Bob Brown', '+1212345678'],
    ['carol.white@example.com', 'Carol White', '+1098765432'],
    ['wrong_email_format', 'Carol White', '+1098765432'],  # Invalid email
]

# Create a DataFrame
df = pd.DataFrame(data, columns=['Column1', 'Column2', 'Column3'])

# Write the DataFrame to an Excel file
file_path = 'sample_unordered_data.xlsx'
df.to_excel(file_path, index=False)

print(f"Excel sheet created at: {file_path}")


Excel sheet created at: sample_unordered_data.xlsx


In [7]:
import pandas as pd
import re

# Read the Excel file
file_path = 'sample_unordered_data.xlsx'
df = pd.read_excel(file_path)

# Initialize lists for valid data
names = []
emails = []
phone_numbers = []

# Regular expressions for email, phone numbers, and names
email_regex = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}')
# More strict phone number pattern, focusing on digits and common delimiters
phone_regex = re.compile(r'^\+?\d{1,4}[-.\s]?\(?\d{1,4}\)?[-.\s]?\d{1,4}[-.\s]?\d{1,9}$')
name_regex = re.compile(r'^[A-Za-z\s]+$')

# Function to validate phone number format
def is_valid_phone_number(phone):
    if phone_regex.match(phone):
        # Further validation to exclude numbers with extensions or invalid formats
        phone = re.sub(r'[^\d]', '', phone)  # Remove all non-digit characters
        return len(phone) >= 10 and len(phone) <= 15  # Adjust based on valid length ranges
    return False

# Function to identify and extract valid data
def extract_valid_data(row):
    email, phone, name = None, None, None
    for cell in row:
        if isinstance(cell, str):
            cell = cell.strip()
            if re.match(email_regex, cell):
                email = cell
            elif is_valid_phone_number(cell):
                phone = cell
            elif re.match(name_regex, cell):
                name = cell
    if email and phone and name:
        emails.append(email)
        phone_numbers.append(phone)
        names.append(name)

# Loop through the DataFrame rows
for _, row in df.iterrows():
    extract_valid_data(row)

# Remove duplicates
names = list(set(names))
emails = list(set(emails))
phone_numbers = list(set(phone_numbers))

# Output valid lists
print("Valid Names:", names)
print("Valid Emails:", emails)
print("Valid Phone Numbers:", phone_numbers)

# Create a DataFrame to save the valid extracted data
max_len = max(len(names), len(emails), len(phone_numbers))
output_df = pd.DataFrame({
    'Names': names + [''] * (max_len - len(names)),
    'Emails': emails + [''] * (max_len - len(emails)),
    'Phone Numbers': phone_numbers + [''] * (max_len - len(phone_numbers))
})

# Save the DataFrame to an Excel file
output_file_path = 'extracted_valid_data.xlsx'
output_df.to_excel(output_file_path, index=False)

print(f"Extracted valid data saved to: {output_file_path}")

Valid Names: ['Sally Mosley', 'Alice Johnson', 'Bob Brown', 'Carol White']
Valid Emails: ['carol.white@example.com', 'alice.johnson@example.com', 'bob.brown@example.com', 'csmith@example.com']
Valid Phone Numbers: ['001-950-804-2214', '+1123456789', '+1098765432', '+1212345678']
Extracted valid data saved to: extracted_valid_data.xlsx
